# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [197]:
# import libraries
import sys
import os
import re
import pickle
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report, f1_score, recall_score, precision_score, accuracy_score, precision_recall_curve, precision_recall_fscore_support, multilabel_confusion_matrix
from sklearn.base import BaseEstimator,TransformerMixin

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\avanjavakam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\avanjavakam\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\avanjavakam\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\avanjavakam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [243]:
# load data from database
engine = create_engine('sqlite:///disaster_response.db')
df = pd.read_sql_table('messages', engine)


# Given value 2 in the related field are neglible so it could be error. Replacing 2 with 1 to consider it a valid response.
# Alternatively, we could have assumed it to be 0 also. In the absence of information I have gone with majority class.
df['related']=df['related'].map(lambda x: 1 if x == 2 else x)

#Remove child alone as it has all zeros only
df = df.drop(['child_alone'],axis=1)

# separate feature and target into X and Y
X = df['message']
Y = df.drop(['message','id','original','genre'], axis=1)

Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [200]:
def tokenize(text):
    
    #removing punctuations, lowering the case and stripping leading and trailing space
    text = re.sub(r"[^a-zA-Z0-9]"," ", text.lower().strip())
    
    #tokenize text
    tokens = word_tokenize(text)

    #lemmatize initialization
    lemmatizer = WordNetLemmatizer()

    #lemmatize and store clean tokens
    tokens = [lemmatizer.lemmatize(w).lower().strip() for w in tokens]

    #return cleaned tokens
    return tokens




### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [201]:
# pipeline for features, count vectorizer, tdidf transformer and classifier
pipeline = Pipeline([
        ('features', FeatureUnion([
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer = tokenize)),
                ('tdidf', TfidfTransformer()),
            
            ]))

        ])),
        ('clf', MultiOutputClassifier(estimator = LogisticRegression(random_state=42, verbose = 3)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [229]:
#splitting X, Y into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y)

y_test.values


array([[1, 0, 0, ..., 1, 0, 0],
       [1, 1, 0, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 0]], dtype=int64)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [203]:
# fitting the classifier pipeline on training data
pipeline.fit(X_train, y_train)

# predicting the test data
y_pred = pipeline.predict(X_test)
y_pred



c:\Users\avanjavakam\.conda\envs\datascience\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_job

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 1],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [205]:
# printing precision, recall, f1 score
print(classification_report(y_test.values, y_pred, target_names = Y.columns.values))

                        precision    recall  f1-score   support

               related       0.84      0.96      0.90      5014
               request       0.82      0.56      0.66      1112
                 offer       0.00      0.00      0.00        23
           aid_related       0.77      0.66      0.71      2723
          medical_help       0.71      0.16      0.26       527
      medical_products       0.83      0.21      0.33       326
     search_and_rescue       0.83      0.05      0.10       182
              security       0.00      0.00      0.00       108
              military       0.81      0.12      0.22       208
                 water       0.78      0.50      0.61       410
                  food       0.85      0.61      0.71       741
               shelter       0.83      0.42      0.56       566
              clothing       0.65      0.15      0.25        86
                 money       0.75      0.06      0.12       144
        missing_people       0.00      

c:\Users\avanjavakam\.conda\envs\datascience\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\avanjavakam\.conda\envs\datascience\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\avanjavakam\.conda\envs\datascience\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 6. Improve your model
Use grid search to find better parameters. 

In [206]:
# parameters for logistic regression classifier
parameters = {
    'clf__estimator__penalty': ['l1', 'l2', 'elasticnet'],
    'clf__estimator__class_weight': ['balanced',None],
    'clf__estimator__solver': ['sag', 'saga']

}

# using Grid Search with cross-validation on classifier pipeline along with parameters
cv = GridSearchCV(pipeline, param_grid = parameters, scoring = 'f1_micro', n_jobs = -1)

In [109]:
# fitting the pipeline for best parameters
cv.fit(X_train, y_train)

c:\Users\avanjavakam\.conda\envs\datascience\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\avanjavakam\.conda\envs\datascience\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\avanjavakam\.conda\envs\datascience\lib\site-packages\sklearn\pipeline.py", line 406, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "c:\Users\avanjavakam\.conda\envs\datascience\lib\site-packages\sklearn\multioutput.py", lin

convergence after 47 epochs took 14 seconds


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   13.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   13.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


convergence after 73 epochs took 10 seconds


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    9.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    9.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


...

max_iter reached after 7 seconds


c:\Users\avanjavakam\.conda\envs\datascience\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


convergence after 76 epochs took 8 seconds


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    7.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    7.8s finished


GridSearchCV(estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('text_pipeline',
                                                                        Pipeline(steps=[('vect',
                                                                                         CountVectorizer(tokenizer=<function tokenize at 0x0000029F689EF880>)),
                                                                                        ('tdidf',
                                                                                         TfidfTransformer())]))])),
                                       ('clf',
                                        MultiOutputClassifier(estimator=LogisticRegression(random_state=42,
                                                                                           verbose=3)))]),
             n_jobs=-1,
             param_grid={'clf__estimator__class_weight': ['balanced', None],
                         'clf__estimator__penalty': ['l1', 'l2', 'elasticnet'],
                         'clf__estimator__solver': ['sag', 'saga']},
             scoring='f1_micro')

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [110]:
# Testing the model with best parameters
y_pred = cv.predict(X_test)


In [111]:
# printing precision, recall and f1 score
print(classification_report(y_test.values, y_pred, target_names = Y.columns.values))

                        precision    recall  f1-score   support

               related       0.85      0.95      0.90      5014
               request       0.80      0.57      0.66      1138
                 offer       0.00      0.00      0.00        26
           aid_related       0.78      0.68      0.72      2722
          medical_help       0.63      0.26      0.36       527
      medical_products       0.69      0.30      0.42       335
     search_and_rescue       0.79      0.19      0.30       180
              security       1.00      0.01      0.02       112
              military       0.68      0.34      0.45       214
                 water       0.80      0.63      0.71       427
                  food       0.83      0.68      0.75       739
               shelter       0.79      0.57      0.66       568
              clothing       0.72      0.47      0.57       104
                 money       0.62      0.25      0.36       130
        missing_people       0.65      

c:\Users\avanjavakam\.conda\envs\datascience\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\avanjavakam\.conda\envs\datascience\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\avanjavakam\.conda\envs\datascience\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [210]:
# Building a custom transformer to extract the starting verb of a sentence
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    """
    Starting Verb Extractor class
    
    This class extract the starting verb of the message,
    creating a new feature for the ML classifier
    """
    def starting_verb(self, text):
        
        # sentence tokenizing the text
        sentence_list = nltk.sent_tokenize(text)
        
        
        for sentence in sentence_list:
            # tokenize each sentence and tag parts of speech
            pos_tags = nltk.pos_tag(tokenize(sentence))
            
            try:
                # extract the first word and its tagged part of speech
                first_word, first_tag = pos_tags[0]
                #if it's a verb, return true
                if first_tag in ['VB','VBP'] or first_word == 'RT':
                    return float(1)
            except:
                # In all other cases, return false
                return float(0)

    #Given it is a transformer, return self
    def fit(self, X, y=None):
        return self

    # defining transform
    def transform(self, X):
        # applying the starting_verb function to add this feature
        X_with_starting_verb = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_with_starting_verb).dropna(inplace = True)


StartingVerbExtractor()

In [217]:
# Improving our existing Pipeline with added feature - StartingVerbExtractor and an ensemble classifier
pipeline_starting_verb_ext = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb_transformer', StartingVerbExtractor())
        ])),

        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    ])

In [213]:
#splitting data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y)

#fitting new pipeline with added feature
pipeline_starting_verb_ext.fit(X_train, y_train)

#predicting
y_pred = pipeline_starting_verb_ext.predict(X_test)
y_pred

c:\Users\avanjavakam\.conda\envs\datascience\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [214]:
#reporting classification metrics
print(classification_report(y_test.values, y_pred, target_names = Y.columns.values))

                        precision    recall  f1-score   support

               related       0.83      0.95      0.88      5021
               request       0.76      0.53      0.62      1126
                 offer       0.00      0.00      0.00        32
           aid_related       0.76      0.58      0.66      2760
          medical_help       0.63      0.28      0.39       524
      medical_products       0.66      0.32      0.43       332
     search_and_rescue       0.60      0.16      0.26       179
              security       0.33      0.05      0.08       128
              military       0.58      0.29      0.38       221
                 water       0.73      0.65      0.69       429
                  food       0.82      0.66      0.73       705
               shelter       0.77      0.52      0.62       578
              clothing       0.75      0.44      0.55       109
                 money       0.56      0.30      0.40       158
        missing_people       0.60      

c:\Users\avanjavakam\.conda\envs\datascience\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\avanjavakam\.conda\envs\datascience\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [218]:
# parameters for adaboost classifier
parameters = {
    'clf__estimator__n_estimators': [10, 50, 100],
    'clf__estimator__learning_rate': [0.001, 0.1, 1]
}

# passing new pipeline and parameters for grid search
cv = GridSearchCV(pipeline_starting_verb_ext, param_grid = parameters, scoring = 'f1_micro', n_jobs = -1)

#fitting the new pipeline for best parameters
cv.fit(X_train, y_train)

c:\Users\avanjavakam\.conda\envs\datascience\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


GridSearchCV(estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('text_pipeline',
                                                                        Pipeline(steps=[('vect',
                                                                                         CountVectorizer(tokenizer=<function tokenize at 0x0000029F037EEB00>)),
                                                                                        ('tfidf',
                                                                                         TfidfTransformer())])),
                                                                       ('starting_verb_transformer',
                                                                        StartingVerbExtractor())])),
                                       ('clf',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier()))]),
             n_jobs=-1,
             param_grid={'clf__estimator__learning_rate': [0.001, 0.1, 1],
                         'clf__estimator__n_estimators': [10, 50, 100]},
             scoring='f1_micro')

In [224]:
# predicting with best parameters
y_pred = cv.predict(X_test)

{'clf__estimator__learning_rate': 1, 'clf__estimator__n_estimators': 100}

In [220]:
# printing precision, recall, f1 score
print(classification_report(y_test.values, y_pred, target_names = Y.columns.values))

                        precision    recall  f1-score   support

               related       0.84      0.94      0.89      5021
               request       0.76      0.56      0.64      1126
                 offer       0.00      0.00      0.00        32
           aid_related       0.77      0.61      0.68      2760
          medical_help       0.61      0.27      0.38       524
      medical_products       0.61      0.33      0.42       332
     search_and_rescue       0.47      0.17      0.25       179
              security       0.38      0.09      0.14       128
              military       0.60      0.33      0.42       221
                 water       0.72      0.64      0.68       429
                  food       0.81      0.64      0.72       705
               shelter       0.75      0.53      0.62       578
              clothing       0.73      0.45      0.56       109
                 money       0.49      0.25      0.33       158
        missing_people       0.54      

c:\Users\avanjavakam\.conda\envs\datascience\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\avanjavakam\.conda\envs\datascience\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 9. Export your model as a pickle file

In [222]:
pickle.dump(cv, open('disaster_response_cv_classifier.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

'sqlite:///sss'